In [2]:
import json
import numpy as np
import random
import pickle

In [19]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [20]:
def to_np(x,per):
    x_np = {}
    for k,v in x.items():
        if per is not None:
            if isinstance(per,int):
                if per < len(v):
                    random.shuffle(v)
                    v = v[:per]
            elif isinstance(per,float):
                v = v[:int(len(v)*per)]
        if k =='y':
            x_np[k] = sigmoid(np.array(v))
        else:
            x_np[k] = np.array(v)
        
    return x_np

In [23]:
def load_neo_datasets(file_name):
    data = json.load(open(file_name))
    train_data = to_np(data['train'],2000)
    val_data = to_np(data['test'],500)
    test_data = to_np(data['ood'],500)
    w = data['w']
    return train_data, val_data, test_data,w

In [22]:
for dataset_name in ['cross_species','GSM','Norman2019_prep','pachter']:
    train_data, val_data, test_data,w = load_neo_datasets(f"datasets/{dataset_name}_new.json")
    with open(f"datasets/{dataset_name}.pkl","wb") as f:
        pickle.dump([train_data, val_data, test_data,w],f)

In [16]:
val_data['x'].shape[0]/20

195.7

In [19]:
res_val = []
res_test = []
ptr = ['-','TR','PTR']
for dataset_name in ['cross_species','GSM','Norman2019_prep','pachter']:
    res_val.append(f"{dataset_name}")
    res_test.append(f"{dataset_name}")
    for p in [0,1,2]:
        # tmp_val = f"{dataset_name} & {p}"
        # tmp_test = f"{dataset_name} & {p}"
        tmp_val = f"& {ptr[p]}"
        tmp_test = f"& {ptr[p]}"
        with open(f'saved_models/TransTEE/{dataset_name}/tr10/trb2.0/dob2.0/p{p}0.5result.json','r') as f:
            data = json.load(f)
        val_dict = data['val']
        ood_dict = data['test']
        
        for met in ['mise','dpe','pe','ate']:
            x_mean = np.mean(val_dict[met])
            x_var = np.std(val_dict[met])
            tmp_str = f"$ {x_mean:.4f} \pm {x_var:.4f} $"
            tmp_val=tmp_val+" & "+tmp_str
        for met in ['mise','dpe','pe','ate']:
            x_mean = np.mean(ood_dict[met])
            x_var = np.std(ood_dict[met])
            tmp_str = f"$ {x_mean:.4f} \pm {x_var:.4f} $"
            tmp_test=tmp_test+" & "+tmp_str
        tmp_val += "\\\\"
        tmp_test += "\\\\"
        res_val.append(tmp_val)
        res_test.append(tmp_test)

In [20]:
with open('latex.txt','w') as f:
    for r in res_val:
        f.write(r+'\n')
    for r in res_test:
        f.write(r+'\n')